# Basics of Spark on HDInsight

<a href="http://spark.apache.org/" target="_blank">Apache Spark</a> is an open-source parallel processing framework that supports in-memory processing to boost the performance of big-data analytic applications. When you provision a Spark cluster in HDInsight, you provision Azure compute resources with Spark installed and configured. The data to be processed is stored in Azure Blob storage (WASB).

![Spark on HDInsight](https://mysstorage.blob.core.windows.net/notebookimages/overview/SparkArchitecture.png "Spark on HDInsight")

Now that you have created a Spark cluster, let us understand some basics of working with Spark on HDInsight. For detailed discussion on working with Spark, see [Spark Programming Guide](http://spark.apache.org/docs/2.0.0/sql-programming-guide.html).

----------
## Notebook setup

When using PySpark kernel notebooks on HDInsight, there is no need to create a SparkContext or a SparkSession; a SparkSession which has the SparkContext is created for you automatically when you run the first code cell, and you'll be able to see the progress printed. The contexts are created with the following variable names:
- SparkSession (spark)

To run the cells below, place the cursor in the cell and then press **SHIFT + ENTER**.

----------

## What is an RDD?

Big Data applications rely on iterative, distributed computing for faster processing of large data sets. To distribute data processing over multiple jobs, the data is typically reused or shared across jobs. To share data between  existing distributed computing systems you need to store data in some intermediate stable distributed store such as HDFS. This makes the overall computations of jobs slower.

**Resilient Distributed Datasets** or RDDs address this by enabling fault-tolerant, distributed, in-memory computations.

----------

## How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from the sample data files available in the storage container associated with your Spark cluster. One such sample data file is available on the cluster at `wasb:///example/data/fruits.txt`. 

In [1]:
fruits = spark.sparkContext.textFile('wasb:///example/data/fruits.txt')
yellowThings = spark.sparkContext.textFile('wasb:///example/data/yellowthings.txt')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1597762721694_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

For more examples on how to create RDDs see the following notebooks available with your Spark cluster:

* Read and write data from Azure Storage Blobs (WASB)
* Read and write data from Hive tables

----------

## What are RDD operations?
RDDs support two types of operations: transformations and actions.

* **Transformations** create a new dataset from an existing one. Transformations are lazy, meaning that no transformation is executed until you execute an action.
* **Actions** return a value to the driver program after running a computation on the dataset.

### RDD transformations
Following are examples of some of the common transformations available. For a detailed list, see [RDD Transformations](https://spark.apache.org/docs/2.0.0/programming-guide.html#transformations)

Run some transformations below to understand this better. Place the cursor in the cell and press **SHIFT + ENTER**.

In [2]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

In [5]:
fruits.collect()

[u'apple', u'banana', u'canary melon', u'grape', u'lemon', u'orange', u'pineapple', u'strawberry']

In [6]:
fruitsReversed.collect()

[u'elppa', u'ananab', u'nolem yranac', u'eparg', u'nomel', u'egnaro', u'elppaenip', u'yrrebwarts']

In [7]:
# filter
shortFruits = fruits.filter(lambda fruit: len(fruit) <= 5)
shortFruits.collect()

[u'apple', u'grape', u'lemon']

In [8]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))
characters.collect()

[u'a', u'p', u'p', u'l', u'e', u'b', u'a', u'n', u'a', u'n', u'a', u'c', u'a', u'n', u'a', u'r', u'y', u' ', u'm', u'e', u'l', u'o', u'n', u'g', u'r', u'a', u'p', u'e', u'l', u'e', u'm', u'o', u'n', u'o', u'r', u'a', u'n', u'g', u'e', u'p', u'i', u'n', u'e', u'a', u'p', u'p', u'l', u'e', u's', u't', u'r', u'a', u'w', u'b', u'e', u'r', u'r', u'y']

In [9]:
# union
fruitsAndYellowThings = fruits.union(yellowThings)
fruitsAndYellowThings.collect()

[u'apple', u'banana', u'canary melon', u'grape', u'lemon', u'orange', u'pineapple', u'strawberry', u'banana', u'bee', u'butter', u'canary melon', u'gold', u'lemon', u'pineapple', u'sunflower']

In [10]:
# intersection
yellowFruits = fruits.intersection(yellowThings)
yellowFruits.collect()

[u'lemon', u'canary melon', u'banana', u'pineapple']

In [11]:
# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
distinctFruitsAndYellowThings.collect()

[u'orange', u'butter', u'lemon', u'grape', u'canary melon', u'strawberry', u'banana', u'apple', u'sunflower', u'gold', u'bee', u'pineapple']

In [13]:
# groupByKey
yellowThingsByFirstLetter = yellowThings.map(lambda thing: (thing[0], thing)).groupByKey()
yellowThingsByFirstLetter

PythonRDD[30] at RDD at PythonRDD.scala:52

In [14]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)
numFruitsByLength.collect()

[(10, 1), (12, 1), (6, 2), (9, 1), (5, 3)]

### RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/2.0.0/programming-guide.html#actions).

Run some transformations below to understand this better. Place the cursor in the cell and press **SHIFT + ENTER**.

In [15]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
fruitsArray

[u'apple', u'banana', u'canary melon', u'grape', u'lemon', u'orange', u'pineapple', u'strawberry']

In [16]:
# count
numFruits = fruits.count()
numFruits

8

In [17]:
# take
first3Fruits = fruits.take(3)
first3Fruits

[u'apple', u'banana', u'canary melon']

In [18]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
letterSet

set([u'a', u' ', u'c', u'b', u'e', u'g', u'i', u'm', u'l', u'o', u'n', u'p', u's', u'r', u't', u'w', u'y'])

> **IMPORTANT**: Another important RDD action is saving the output to a file. See the **Read and write data from Azure Storage Blobs (WASB)** notebook for more information.

----------

## What is a dataframe?

The `pyspark.sql` library provides an alternative API for manipulating structured datasets, known as "dataframes". (Dataframes are not a Spark-specific concept but `pyspark` provides its own dedicated dataframe library.) These are different from RDDs, but you can convert an RDD into a dataframe or vice-versa, if required.

See [Spark SQL and DataFrame Guide](https://spark.apache.org/docs/2.0.0/sql-programming-guide.html#datasets-and-dataframes) for more information.

### How do I make a dataframe?

You can load a dataframe directly from an input data source. See the following notebooks included with your Spark cluster for more information.

* Read and write data from Azure Storage Blobs (WASB)
* Read and write data from Hive tables

You can also create a dataframe from a CSV file as shown below.

In [19]:
df = spark.read.csv('wasb:///HdiSamples/HdiSamples/SensorSampleData/building/building.csv', header=True, inferSchema=True)

### Dataframe operations

Run the cells below to see examples of some of the the operations that you can perform on dataframes.

In [20]:
# show the content of the dataframe
df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [21]:
# Print the dataframe schema in a tree format
df.printSchema()

root
 |-- BuildingID: integer (nullable = true)
 |-- BuildingMgr: string (nullable = true)
 |-- BuildingAge: integer (nullable = true)
 |-- HVACproduct: string (nullable = true)
 |-- Country: string (nullable = true)

In [22]:
# Create an RDD from the dataframe
dfrdd = df.rdd
dfrdd.take(3)

[Row(BuildingID=1, BuildingMgr=u'M1', BuildingAge=25, HVACproduct=u'AC1000', Country=u'USA'), Row(BuildingID=2, BuildingMgr=u'M2', BuildingAge=27, HVACproduct=u'FN39TG', Country=u'France'), Row(BuildingID=3, BuildingMgr=u'M3', BuildingAge=28, HVACproduct=u'JDNS77', Country=u'Brazil')]

In [23]:
dfrdd = df.rdd
dfrdd.take(3)

[Row(BuildingID=1, BuildingMgr=u'M1', BuildingAge=25, HVACproduct=u'AC1000', Country=u'USA'), Row(BuildingID=2, BuildingMgr=u'M2', BuildingAge=27, HVACproduct=u'FN39TG', Country=u'France'), Row(BuildingID=3, BuildingMgr=u'M3', BuildingAge=28, HVACproduct=u'JDNS77', Country=u'Brazil')]

In [24]:
# Retrieve a given number of rows from the dataframe
df.limit(3).show()

+----------+-----------+-----------+-----------+-------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|Country|
+----------+-----------+-----------+-----------+-------+
|         1|         M1|         25|     AC1000|    USA|
|         2|         M2|         27|     FN39TG| France|
|         3|         M3|         28|     JDNS77| Brazil|
+----------+-----------+-----------+-----------+-------+

In [25]:
df.limit(3).show()

+----------+-----------+-----------+-----------+-------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|Country|
+----------+-----------+-----------+-----------+-------+
|         1|         M1|         25|     AC1000|    USA|
|         2|         M2|         27|     FN39TG| France|
|         3|         M3|         28|     JDNS77| Brazil|
+----------+-----------+-----------+-----------+-------+

In [26]:
# Retrieve specific columns from the dataframe
df.select('BuildingID', 'Country').limit(3).show()

+----------+-------+
|BuildingID|Country|
+----------+-------+
|         1|    USA|
|         2| France|
|         3| Brazil|
+----------+-------+

In [27]:
# Use GroupBy clause with dataframe 
df.groupBy('HVACProduct').count().select('HVACProduct', 'count').show()

+-----------+-----+
|HVACProduct|count|
+-----------+-----+
|    ACMAX22|    4|
|     AC1000|    4|
|     JDNS77|    4|
|     FN39TG|    4|
|     GG1919|    4|
+-----------+-----+

> **IMPORTANT**: Many of the methods available on normal RDDs are also available on dataframes. For example, `distinct`, `count`, `collect`, `filter`, `map`, and `take` are all methods on dataframes as well as on RDDs.

-------

## Spark SQL and dataframes

You can also run SQL queries over dataframes once you register them as temporary tables within the SparkSession. Run the snippet below to see an example.

In [28]:
# Register the dataframe as a temporary table called HVAC
df.registerTempTable('HVAC')

In [29]:
%%sql
SELECT * FROM HVAC WHERE BuildingAge >= 10

In [30]:
%%sql 
SELECT BuildingID, Country FROM HVAC LIMIT 3